In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
import cv2
import math

In [2]:
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image

In [3]:
from motrackers import CentroidTracker
from motrackers.utils import draw_tracks
tracker = CentroidTracker(max_lost=0, tracker_output_format='mot_challenge')

In [4]:
def estimateSpeed(location1, location2):
    d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))

    ppm = 8.8
    d_meters = d_pixels / ppm
    fps = 18
    speed = d_meters * fps * 3.6
    return speed

In [5]:
cap = cv2.VideoCapture('test1.mp4')

In [6]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

In [7]:
# ## Object detection imports
# Here are the imports from the object detection module.
from utils import label_map_util
from utils import visualization_utils as vis_util

In [8]:
# # Model preparation 
MODEL_NAME = 'ckpt_faster_rcnn_inception'

In [9]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

In [10]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('label_map', 'object_detection.pbtxt')
NUM_CLASSES = 4

In [17]:
# ## Load a (frozen) Tensorflow model into memory.
detection_graph = tf.compat.v1.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [12]:
# ## Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [13]:
from motrackers.detectors.detector import Detector

In [14]:
# # Detection
ids = {}

with detection_graph.as_default():
    with tf.compat.v1.Session(graph=detection_graph) as sess:
        
        while cap.isOpened():
            keys = ids.keys()
            flag = [0]*len(keys)
            ret, image = cap.read()
            ops = tf.compat.v1.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in ['num_detections', 'detection_boxes', 'detection_scores','detection_classes', 'detection_masks']:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.compat.v1.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
            # The following processing is only for single image
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])

                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.compat.v1.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                 feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
              'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
          
            i = image.shape
            w = i[0]
            h = i[1]
            
            for i in range(len(output_dict['detection_boxes'])):
                if output_dict['detection_classes'][i]:
                    bb = [int(output_dict['detection_boxes'][i][1]*w), int(output_dict['detection_boxes'][i][0]*h), 
                       int((output_dict['detection_boxes'][i][3]-output_dict['detection_boxes'][i][1])*w),
                       int((output_dict['detection_boxes'][i][2]-output_dict['detection_boxes'][i][0])*h)]
                cv2.rectangle(image, (bb[0], bb[1]), (bb[0] + bb[2], bb[1] + bb[3]), (255,0,0), 2)
                cv2.imshow("image", image)
            a = """vis_util.visualize_boxes_and_labels_on_image_array(
            image,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            line_thickness=12)"""
            

                
            #cv2.imshow('Object detection', cv2.resize(image, (1368,768)))
            if cv2.waitKey(1000) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break
            bbox = []
            conf = []
            cls = []
            for i in range(len(output_dict['detection_boxes'])):
                if output_dict['detection_classes'][i]==2:
                    box = [output_dict['detection_boxes'][i][1]*w, output_dict['detection_boxes'][i][0]*h, 
                       (output_dict['detection_boxes'][i][3]-output_dict['detection_boxes'][i][1])*w,
                       (output_dict['detection_boxes'][i][2]-output_dict['detection_boxes'][i][0])*h]
                    bbox.append(box)
                    conf.append(output_dict['detection_scores'][i])
                    cls.append(output_dict['detection_classes'][i])
          
            #print(bbox)
            c = [output_dict['detection_scores'][i]]
            cl = [output_dict['detection_classes'][i]]
            
            #output_tracks = tracker.update(bbox, c, cl)
            output_tracks = tracker.update(bbox, conf, cls)
            
            bbox_t = []
          
            updated_image = draw_tracks(image, output_tracks)
            cv2.imshow("image1", updated_image)
            
            for track in output_tracks:
                frame, id, bb_left, bb_top, bb_width, bb_height, confidence, x, y, z = track
                assert len(track) == 10
                bb = [int(bb_left), int(bb_top), int(bb_width), int(bb_height)]
                
                bb1 = [bb_left, bb_top, bb_width, bb_height]
                cv2.rectangle(updated_image, (bb[0], bb[1]), (bb[0] + bb[2], bb[1] + bb[3]), (0,255,0), 2)
                cv2.imshow('image2',updated_image)
                 
                f = 0
                keys = ids.keys()
                keys = list(keys)
                c = 0
                for i in keys:
                    if i==str(id):
                        f = 1
                        s = estimateSpeed(ids[str(id)], bb1)
                        print("Speed: ", id, s)
                        flag[c] = 1
                        ids.update({str(id):bb1})
                        break
                        
                    c = c +1
                        
                if f==0:
                    ids[str(id)] = bb1
                    flag.append(0)
                    
                #print(box)
                bbox_t.append(bb1)
            n = len(keys)
            i = 0
            while i<n:
                if flag[i]==0:
                    ids.pop(keys[i])
                    n-=1
                i+=1
            i = 0
            

Metal device set to: Apple M1 Pro


2022-06-10 23:47:35.477495: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-10 23:47:35.477623: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-06-10 23:47:35.530006: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-06-10 23:47:35.548331: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-10 23:47:35.548762: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-10 23:47:35.848969: W tensorflow/core/grappler/costs/op_level_cost_est

Speed:  3 21.471410086965275
Speed:  4 5179.700623214717
Speed:  5 31.939672426269933
Speed:  2 37.60384328395461
Speed:  3 41.23039359090974
Speed:  4 5168.2483672592325
Speed:  0 16.940527026117227
Speed:  1 29.539118940639785
Speed:  2 13.312831542640646
Speed:  3 10.047347977157209
Speed:  4 4.0931958850161685
Speed:  0 13.917860615882702
Speed:  1 199.09595861654057
Speed:  2 20.432212694989403
Speed:  3 7.797810940078758
Speed:  4 17.974561581482725
Speed:  0 19.688334501787956
Speed:  1 89.24344993842244
Speed:  2 7.921485874975833
Speed:  3 9.026323977123978
Speed:  4 23.156184634481175
Speed:  7 2425.203329199251
Speed:  0 7.224413196790913
Speed:  1 83.99996789342725
Speed:  2 5.034081948280527
Speed:  3 7.050533828727883
Speed:  4 32.813710165341924
Speed:  7 3801.175280819271
Speed:  0 8.271297712241994
Speed:  1 56.18897110909408
Speed:  2 22.03117541212566
Speed:  3 12.26954842353381
Speed:  4 24.48438486876681
Speed:  0 10.581645028867632
Speed:  1 61.995598505531085
Spe

Speed:  0 2.4349159158252167
Speed:  1 101.59269748067315
Speed:  2 33.166725956668
Speed:  24 81.46579059708877
Speed:  26 633.4871569054285
Speed:  0 10.552072855283601
Speed:  1 23.74678859178056
Speed:  2 52.4780094628882
Speed:  24 81.36899615946494
Speed:  26 632.2087416804059
Speed:  0 14.070099756701582
Speed:  1 77.99911957478469
Speed:  2 25.56999761577976
Speed:  24 38.724039083420784
Speed:  0 15.068467797491051
Speed:  1 115.56284669613855
Speed:  2 36.416612590948915
Speed:  24 30.72583477303041
Speed:  0 1.2182360963281238
Speed:  1 5.921662070534446
Speed:  2 21.691510358280347
Speed:  24 36.62224564635469
Speed:  29 2.549738274083986
Speed:  0 8.432125799848489
Speed:  1 68.90543850985439
Speed:  2 23.70434016567534
Speed:  24 102.36501464430512
Speed:  29 83.54249371134979
Speed:  0 3.3743807821680716
Speed:  1 53.74917463822798
Speed:  2 24.43022687539486
Speed:  24 45.10576867916532
Speed:  28 6.407110546246313
Speed:  29 6.1073946521272715
Speed:  0 4.9497495624603

Speed:  0 8.852733513709962
Speed:  1 174.44703535600144
Speed:  2 30.981318428491242
Speed:  29 22.024322334583015
Speed:  46 21.75066913431734
Speed:  0 16.253693066347928
Speed:  1 158.658183704723
Speed:  2 9.136833355911042
Speed:  29 36.30990422325275
Speed:  46 13.977638300831781
Speed:  0 13.13010720401048
Speed:  1 42.80516058360632
Speed:  2 15.680626118439433
Speed:  46 23.487523650237982
Speed:  50 20.746582637755953
Speed:  0 2.845440688484944
Speed:  1 116.02267218643689
Speed:  2 24.50444605938337
Speed:  46 12.231148686031613
Speed:  50 83.59731658754058
Speed:  0 14.937918723118363
Speed:  1 192.11549729483852
Speed:  2 9.446472015472482
Speed:  46 41.74563055808005
Speed:  50 31.617797116483523
Speed:  0 15.454141033490805
Speed:  1 87.2002417347559
Speed:  2 37.69093983565918
Speed:  46 64.9812277209862
Speed:  50 45.857376933090116
Speed:  51 3806.282299028097
Speed:  0 32.37335796912486
Speed:  1 36.24427118235808
Speed:  2 54.53339347484365
Speed:  46 581.99785801

...

Speed:  0 3.802023344008273
Speed:  158 28.859206891929624
Speed:  180 22.81483170154241
Speed:  188 31.066819158631937
Speed:  190 22.45076256786503
Speed:  199 41.65558831585024
Speed:  200 38.52813348473039
Speed:  202 132.87848098593375
Speed:  0 17.279123431363658
Speed:  158 10.165227880954971
Speed:  180 26.958014202526467
Speed:  188 3.1411859537400786
Speed:  190 12.973326573118596
Speed:  199 35.54500922500322
Speed:  200 157.57397608093032
Speed:  202 30.446201013326682
Speed:  0 10.513555132088282
Speed:  158 29.009237703994337
Speed:  180 22.981810056650232
Speed:  188 14.860238329769784
Speed:  190 7.823091674864762
Speed:  199 34.997211954254226
Speed:  200 10.235064051455534
Speed:  202 3.7396777795677596
Speed:  204 5215.017319772239
Speed:  0 22.45563634895097
Speed:  158 33.74573595562233
Speed:  180 59.69044280618459
Speed:  188 25.786252627585633
Speed:  190 16.482538018008146
Speed:  199 31.42387660830073
Speed:  200 9.878655778789872
Speed:  204 176.6344025469468

Speed:  0 29.274176223555926
Speed:  158 97.41932568507931
Speed:  188 17.7789412940283
Speed:  204 82.61315494916894
Speed:  210 66.72680560559611
Speed:  216 27.108350831044955
Speed:  218 59.81809682305471
Speed:  0 14.817612503673514
Speed:  158 59.255946766246446
Speed:  188 82.01332707553965
Speed:  204 910.8015764668294
Speed:  210 96.90463681838717
Speed:  216 23.41007525187661
Speed:  218 1456.036315166679
Speed:  0 39.292584248108454
Speed:  158 25.196207219904117
Speed:  188 24.593046402524575
Speed:  204 1031.7891048333156
Speed:  210 124.3683969295008
Speed:  216 2732.4545812700253
Speed:  218 1404.0928989215681
Speed:  0 24.430535046306684
Speed:  158 41.08955383300781
Speed:  188 24.09062759243202
Speed:  204 11.684351932567106
Speed:  210 57.568501061977614
Speed:  216 1138.381397047856
Speed:  218 1137.3590128357466
Speed:  0 13.598969827843721
Speed:  158 87.79428985725733
Speed:  188 76.57861423933902
Speed:  204 31.934581168271183
Speed:  210 29.999605868622623
Spee

Speed:  0 19.78365892114728
Speed:  188 56.31070802212367
Speed:  204 11.125904430042612
Speed:  236 137.71139480659335
Speed:  238 68.7882917893654
Speed:  239 12.46704294378997
Speed:  0 39.9718829032016
Speed:  204 1.6207955100319602
Speed:  236 48.697111319164165
Speed:  238 51.36895682005436
Speed:  239 449.1233654067206
Speed:  0 59.617596847633
Speed:  204 22.956307151100848
Speed:  236 34.76216461886263
Speed:  238 44.72904490082852
Speed:  239 1222.7143232663002
Speed:  0 14.057863664012658
Speed:  204 22.7456318248402
Speed:  236 2439.006620750473
Speed:  238 1966.6975357421684
Speed:  239 1143.8177768393305
Speed:  0 44.713494297722946
Speed:  204 31.370960582386363
Speed:  236 2429.050915154332
Speed:  238 1972.8043583108915
Speed:  239 39.96132747649014
Speed:  0 18.546102907915927
Speed:  204 20.472585504705258
Speed:  236 59.256794042115715
Speed:  238 53.498242438118545
Speed:  239 116.04605499949311
Speed:  242 35.704053057572374
Speed:  0 18.782554034289785
Speed:  20

Speed:  243 15.763554756002137
Speed:  257 205.7250240278881
Speed:  264 213.61779791209145
Speed:  265 16.979611984461343
Speed:  272 74.08418318372891
Speed:  275 1109.5861218402904
Speed:  276 7.327189083203404
Speed:  243 17.6113529207268
Speed:  257 3127.5028470624125
Speed:  264 141.08451643654772
Speed:  265 58.907151411665396
Speed:  272 68.97676322250966
Speed:  275 17.613278941029648
Speed:  276 16.39015317101308
Speed:  243 19.986300351262567
Speed:  257 8.709145807629977
Speed:  264 51.36405937361708
Speed:  265 27.92188648325579
Speed:  272 39.02044343348068
Speed:  275 482.65846995618614
Speed:  276 94.8469639133572
Speed:  277 2128.4606479623253
Speed:  243 6.383698734876812
Speed:  257 45.02692876711882
Speed:  264 57.69518633580859
Speed:  265 3.330423022693458
Speed:  272 513.5905123064955
Speed:  275 3732.330549558183
Speed:  276 56.35525750865159
Speed:  278 18.202528871647885
Speed:  243 14.540065534909322
Speed:  257 6.335398521821955
Speed:  264 16.12435482406811

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'